# First try

In [7]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth

In [25]:
def encode(df, attributes):
    for attribute in attributes:
        df[attribute] = df[attribute].astype('category')
        df[attribute] = df[attribute].cat.codes
    return df

In [24]:
def get_transaction_table(df):
    df_encoded = encode(df.copy(), ['Watcher', 'videoCategoryLabel', 'definition'])
    return df_encoded.values 

In [26]:
path='dataset2.csv'
df = pd.read_csv(path)
df

,Watcher,videoCategoryId,videoCategoryLabel,definition
0,Billy,29,Nonprofits & Activism,hd
1,Leila,22,People & Blogs,sd
2,Billy,22,People & Blogs,sd
3,Mark,24,Entertainment,hd
4,Billy,24,Entertainment,hd
...,...,...,...,...
110,Jeff,24,Entertainment,hd
111,Billy,26,Howto & Style,sd
112,Clark,28,Science & Technology,hd
113,Billy,24,Entertainment,sd


In [86]:
transaction_table = get_transaction_table(df)

In [ ]:
df.Watcher.value_counts()

In [ ]:
'''for att in list(df.data.keys()):
        c.append({k: v / len(df.data.get(att)) for k, v in df.count(att).items()})
    l = [{k: v for k, v in i.items() if v > min_support} for i in c if i]
    temp = [{df.encode_dict[att][k]: v for k, v in i.items()} for i, att in zip(c, list(df.data.keys()))]
    l.append([t for t in temp if t])
    c = [{k: v for k, v in i.items()} for i in c if i]
    while c:
        print(c[0].keys())
        new_c = [] '''

# Second approch

In [56]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import numpy as np

path = 'dataset2.csv'
dataframe = pd.read_csv(path)
dataframe = dataframe.drop(columns=['videoCategoryId', 'definition'])
dataframe = dataframe.fillna(dataframe.mode().iloc[0])
t = {} 
for k, v in dataframe.values:
    if k in t.keys():
        if v not in t[k]:
            t[k] += [v]
    else:
        t[k] = [v] 
te = TransactionEncoder()
t = [[k] + v for k, v in t.items()]
te_ary = te.fit(t).transform(t)
dataframe = pd.DataFrame(te_ary, columns=te.columns_)

# First Apriori

In [101]:
from DataFrame import DF
import itertools

In [102]:
def get_data(df, norm=False):
    df.missing_values_imputation()
    df.encode_str()
    for att in list(df.data.keys()):
        df.outliers_median_imputation(att)
    if norm:
        for att in list(df.data.keys()):
            df.normalize_minmax(att)
    return [[df.encode_dict[att][df.data[att][i]] for att in list(df.data.keys())] for i in range(len(df.data.get(att)))]
df = DF(remove=True)
transactions = get_data(df)
transactions[:5]

[['Billy', 'Nonprofits & Activism', 'hd'],
 ['Leila', 'People & Blogs', 'sd'],
 ['Billy', 'People & Blogs', 'sd'],
 ['Mark', 'People & Blogs', 'hd'],
 ['Billy', 'People & Blogs', 'hd']]

In [103]:
def generate(l):
    out = []
    for r in range(len(l)):
        out += list(itertools.combinations(l, r+1))
    return out

In [104]:
def Apriori(transactions, keys, min_support = 0.1):
    out = {}
    c = {}
    min_support = min_support * len(transactions)

    for transaction in transactions:
        for item, att in zip(transaction, keys): 
            c[(att, item)] = c[(att, item)] + 1 if (att, item) in c.keys() else 1
    l = {k: v for k, v in c.items() if v >= min_support}
    out.update(l)

    while c:
        c = {}
        for k1, _ in l.items():
            for k2, _ in l.items():
                if (k1[0] != k2[0]):
                    c[((k1[0], k2[0]), (k1[1], k2[1]))] = 0
        
        for transaction in transactions:
            for i in generate(transaction):
                for items in list(c.keys()):
                    if i == items[1]:
                        c[items] += 1
        l = {k: v for k, v in c.items() if v >= min_support}
        out.update(l)
        return l, c, out

In [105]:
support = 0.1
l, c, out = Apriori(transactions, list(df.data.keys()), support) 
out

{('definition', 'hd'): 76,
 ('videoCategoryLabel', 'People & Blogs'): 62,
 ('definition', 'sd'): 39,
 ('videoCategoryLabel', 'Science & Technology'): 30,
 (('videoCategoryLabel', 'definition'), ('People & Blogs', 'hd')): 43,
 (('videoCategoryLabel', 'definition'), ('People & Blogs', 'sd')): 19,
 (('videoCategoryLabel', 'definition'), ('Science & Technology', 'hd')): 16,
 (('videoCategoryLabel', 'definition'), ('Science & Technology', 'sd')): 14}

# New Apriori

In [49]:
from DataFrame import DF
import itertools
import numpy as np

In [51]:
def get_data(df, norm=False):
    df.missing_values_imputation()
    df.encode_str()
    '''for att in list(df.data.keys()):
        df.outliers_median_imputation(att)
    if norm:
        for att in list(df.data.keys()):
            df.normalize_minmax(att)'''
    return [[df.encode_dict[att][df.data[att][i]] for att in list(df.data.keys())] for i in range(len(df.data.get('Watcher')))]

df = DF(remove=['definition', 'videoCategoryId'])
transactions = {}
for k, v in get_data(df):
    if k in transactions.keys():
        if v not in transactions[k]:
            transactions[k] += tuple([v])
    else:
        transactions[k] = tuple([v]) 

In [52]:
transactions = list(transactions.values())

In [53]:
def Apriori(transactions, min_support = 0.2):
    out = {}
    c = {}
    min_support = min_support * len(transactions)

    for transaction in transactions:
        for item in transaction: 
            c[(item,)] = c[(item,)] + 1 if (item,) in c.keys() else 1
    l = {k: v for k, v in c.items() if v >= min_support}
    out.update(l)

    while l:
        c = {}
        for item_set1 in list(l.keys()):
            for item_set2 in list(l.keys()):
                if item_set1 != item_set2: 
                    c[tuple(set(item_set1) | set(item_set2))] = 0   
        for transaction in transactions:
            for item_set in c.keys():
                if (set(item_set).issubset(transaction)):
                    c[item_set] = c[item_set] + 1
        l = {k: v for k, v in c.items() if v >= min_support}
        out.update(l)
    out = sorted(out.items(), key=lambda x:x[1], reverse=True)
    return l, c, out

In [54]:
support = 0.1
l, c, out = Apriori(transactions, support)  
pd.DataFrame(out)

,0,1
0,"(People & Blogs,)",31
1,"(Science & Technology,)",24
2,"(Science & Technology, People & Blogs)",17
3,"(Nonprofits & Activism,)",9
4,"(Science & Technology, Nonprofits & Activism)",9
5,"(Education,)",8
6,"(Nonprofits & Activism, People & Blogs)",8
7,"(Science & Technology, Nonprofits & Activism, ...",8
8,"(Entertainment,)",7
9,"(Education, People & Blogs)",6


In [55]:
result = apriori(dataframe, min_support=0.1, use_colnames=True)
result['support'] *= len(dataframe)
result = result.sort_values(by=['support'], ascending=False)
result

,support,itemsets
4,31.0,(People & Blogs)
5,24.0,(Science & Technology)
12,17.0,"(Science & Technology, People & Blogs)"
3,9.0,(Nonprofits & Activism)
11,9.0,"(Science & Technology, Nonprofits & Activism)"
0,8.0,(Education)
10,8.0,"(Nonprofits & Activism, People & Blogs)"
14,8.0,"(Science & Technology, Nonprofits & Activism, ..."
1,7.0,(Entertainment)
6,6.0,"(Education, People & Blogs)"


In [57]:
association_rules(result, metric="confidence", min_threshold=0.25)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Science & Technology),(People & Blogs),24.0,31.0,17.0,0.708333,0.022849,-727.0,-102.857143
1,(People & Blogs),(Science & Technology),31.0,24.0,17.0,0.548387,0.022849,-727.0,-50.928571
2,(Science & Technology),(Nonprofits & Activism),24.0,9.0,9.0,0.375000,0.041667,-207.0,-12.800000
3,(Nonprofits & Activism),(Science & Technology),9.0,24.0,9.0,1.000000,0.041667,-207.0,inf
4,(Nonprofits & Activism),(People & Blogs),9.0,31.0,8.0,0.888889,0.028674,-271.0,-270.000000
5,(People & Blogs),(Nonprofits & Activism),31.0,9.0,8.0,0.258065,0.028674,-271.0,-10.782609
6,"(Science & Technology, Nonprofits & Activism)",(People & Blogs),9.0,31.0,8.0,0.888889,0.028674,-271.0,-270.000000
7,"(Science & Technology, People & Blogs)",(Nonprofits & Activism),17.0,9.0,8.0,0.470588,0.052288,-145.0,-15.111111
8,"(Nonprofits & Activism, People & Blogs)",(Science & Technology),8.0,24.0,8.0,1.000000,0.041667,-184.0,inf
9,(Science & Technology),"(Nonprofits & Activism, People & Blogs)",24.0,8.0,8.0,0.333333,0.041667,-184.0,-10.500000
